In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Dataset path
dataset_dir = '/kaggle/input/capstone-dataset'

In [ ]:
# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of the data for validation
)

In [ ]:
# Load the training data
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),  # Image size remains at 128x128
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Load the validation data
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Define a deeper CNN model from scratch
model = Sequential()

# First convolutional block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Second convolutional block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Third convolutional block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Fourth convolutional block
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Fifth convolutional block (newly added)
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Flatten the output
model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',
              metrics=['accuracy'])

# Introduce early stopping and learning rate reduction on plateau
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)
]

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=20,  # Increase epochs if needed
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=callbacks
)

In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_accuracy:.2f}')

In [ ]:
# Save the model
model.save('crop_disease_detection_custom_cnn.h5')

**NEW SECTION**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from PIL import Image

# Load the CSV file containing disease information
disease_info_df = pd.read_csv('/kaggle/input/symptoms-solution/detailedcropsolutions.csv')

# Convert the CSV into a dictionary for easy lookup
disease_info = {}
for _, row in disease_info_df.iterrows():
    disease_info[row['Disease']] = {
        "symptom": row['Symptom'],
        "solution": row['Solution'],
        "early_detection_symptom": row['Early Detection Symptom']
    }

# Load the trained model (adjust the path to your model as necessary)
model = tf.keras.models.load_model('/kaggle/input/cnn_model/tensorflow2/default/1/crop_disease_detection_custom_cnn.h5')

# Preprocess the image for prediction
def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((128, 128))  # Resize to match model's expected input size
    img = img.convert('RGB')      # Ensure it's in RGB mode
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict the disease and return symptoms and solutions
def predict_disease(img_path):
    # Preprocess the image
    img_array = preprocess_image(img_path)
    
    # Make a prediction
    predictions = model.predict(img_array)
    
    # Get the class with the highest predicted probability
    predicted_class_index = np.argmax(predictions[0])
    
    # Define the crop and disease class names based on the training set
    class_names = [
        'Corn___Common_Rust', 'Corn___Gray_Leaf_Spot', 'Corn___Northern_Leaf_Blight', 'Corn___Healthy',
        'Cotton___Alternaria', 'Cotton___Healthy', 'Cotton___Mealybug_Attack', 'Cotton___Potassium_Deficiency',
        'Cotton___Thrip', 'Cotton___White_Flies', 'Potato___Early_Blight', 'Potato___Healthy', 'Potato___Late_Blight',
        'Rice___Brown_Spot', 'Rice___Healthy', 'Rice___Leaf_Blast', 'Rice___Neck_Blast', 
        'Sugarcane___Bacterial Blight', 'Sugarcane___Healthy', 'Sugarcane___Red Rot',
        'Wheat___Brown_Rust', 'Wheat___Healthy', 'Wheat___Yellow_Rust'
    ]
    
    predicted_class = class_names[predicted_class_index]
    
    # Get the disease information
    disease_data = disease_info.get(predicted_class, {
        "symptom": "Unknown",
        "solution": "Unknown",
        "early_detection_symptom": "Unknown"
    })
    
    # Output the prediction result
    result = {
        "Predicted Crop and Disease": predicted_class,
        "Symptoms": disease_data['symptom'],
        "Solution": disease_data['solution'],
    }
    
    return result

image_path = '/kaggle/input/test-images-final/Corn Common Rust.jpg'
prediction_output = predict_disease(image_path)
prediction_output
